# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from math import pi
import warnings
import matplotlib.pyplot as plt
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact
import math

# Downloading dataset containing Emissions, GDP and Population

**Warning**: The data for Arsenic (As), Nickel (Ni) and Chromium (Cr) emissions for Switzerland, Luxembourg and Austria were missing so I set these values to 0, because these emissions don't really affect the results

In [ ]:
Emissions_GDP_Population = pd.read_csv('https://raw.githubusercontent.com/AntoineTrabia/Green-Domestic-Product/main/data/Data_for_GrDP.csv')

In [ ]:
Emissions = Emissions_GDP_Population.loc[:,['countries', 'Year', 'Territorial GHG emissions [t]',
       'Residential GHG emissions [t]', 'Transfer GHG emissions [t]',
       'Footprint GHG emissions [t]','As [t]',
       'Pb [t]', 'NOx [t]', 'SOx [t]', 'NH3 [t]', 'PM2.5 [t]', 'PM10 [t]',
       'NMVOC [t]', 'Cd [t]', 'Hg [t]', 'Cr [t]', 'Ni [t]']]

# Marginal Damage Cost of pollutants

## Information on dataset
GHG: Euro per tCO2eq

Health 1: Impact on health from fine particulate matter and ozone
Euro per tonne of emissions of precursors

Health 2: Impact on health from NO2
Euro per tonne of emissions of precursors

Crops: Impact on crops from ozone
Euro per tonne of emissions of precursors

Forests: Impact on forests from ozone
Euro per tonne of emissions of precursors

Buildings: Impact on buildings
Euro per tonne of emissions

Ecosystems: Impact on ecosystems (eutrophication)
Euro per tonne of emissions  of precursors

Heavy metals: Euro per kg of pollutant

## Downloading Dataset

In [ ]:
MDC = pd.read_excel('https://raw.githubusercontent.com/AntoineTrabia/Green-Domestic-Product/main/data/MDC.xlsx')

## Data Treatment

In [ ]:
# We convert the unit into euro per tonne of pollutant for the heavy metals
heavy_metals = ['Pb cost', 'Cd cost', 'Hg cost', 'As cost', 'Ni cost','Cr cost']
for metal in heavy_metals:
  column = metal + ' [Euro/kg]'
  new_column = metal + ' [Euro/t]'
  MDC[column] = 1000*MDC[column]
  MDC.rename(columns={column: new_column},inplace=True)

# External Costs

## Merging Datasets

In [ ]:
Emissions_GDP_Population_MDC = pd.merge(Emissions_GDP_Population, MDC, on=['countries'])
External_costs = Emissions_GDP_Population_MDC.loc[:,['countries','Year']]

## External cost due to impacts of climate change (GHG emissions)

In [ ]:
reporting_method_list = ['Territorial GHG emissions [t]', 'Residential GHG emissions [t]', 'Footprint GHG emissions [t]']
cost_list = ['Low GHG [Euro/t]', 'Central GHG [Euro/t]', 'High GHG [Euro/t]']

for reporting_method in reporting_method_list:
    for cost_scenario in cost_list:
      new_column_name = "GHG: " + reporting_method.split()[0] + " & " + cost_scenario.split()[0] + " scenario [Euro]"
      External_costs[new_column_name] = Emissions_GDP_Population_MDC[reporting_method] * Emissions_GDP_Population_MDC[cost_scenario]

## External costs due to nitrogen oxides (NOx) emissions

In [ ]:
External_costs['Nox (Health - VOLY) [Euro]'] = Emissions_GDP_Population_MDC['NOx [t]'] * (Emissions_GDP_Population_MDC['Health 1: Nox VOLY [Euro/t]'] + Emissions_GDP_Population_MDC['Health 2: NOx VOLY [Euro/t]'])
External_costs['Nox (Health - VSL) [Euro]'] = Emissions_GDP_Population_MDC['NOx [t]'] * (Emissions_GDP_Population_MDC['Health 1: Nox VSL [Euro/t]'] + Emissions_GDP_Population_MDC['Health 2: NOx VSL [Euro/t]'])
External_costs['Nox (Crops, Forests, Buildings & Ecosystems) [Euro]'] = Emissions_GDP_Population_MDC['NOx [t]'] * (Emissions_GDP_Population_MDC['Crops: Nox [Euro/t]'] + Emissions_GDP_Population_MDC['Buildings: Nox [Euro/t]'] + Emissions_GDP_Population_MDC['Ecosystems: Nox [Euro/t]'] + Emissions_GDP_Population_MDC['Forests: Nox [Euro/t]'])
External_costs['Nox (Crops) [Euro]'] = Emissions_GDP_Population_MDC['NOx [t]'] * Emissions_GDP_Population_MDC['Crops: Nox [Euro/t]']
External_costs['Nox (Forests) [Euro]'] = Emissions_GDP_Population_MDC['NOx [t]'] * Emissions_GDP_Population_MDC['Forests: Nox [Euro/t]']
External_costs['Nox (Buildings) [Euro]'] = Emissions_GDP_Population_MDC['NOx [t]'] * Emissions_GDP_Population_MDC['Buildings: Nox [Euro/t]']
External_costs['Nox (Ecosystems) [Euro]'] = Emissions_GDP_Population_MDC['NOx [t]'] * Emissions_GDP_Population_MDC['Ecosystems: Nox [Euro/t]']
External_costs['Nox (Total - VOLY) [Euro]'] = External_costs['Nox (Health - VOLY) [Euro]'] + External_costs['Nox (Crops, Forests, Buildings & Ecosystems) [Euro]']
External_costs['Nox (Total - VSL) [Euro]'] = External_costs['Nox (Health - VSL) [Euro]'] + External_costs['Nox (Crops, Forests, Buildings & Ecosystems) [Euro]']

## External costs due to fine particulate matter (PM2.5) emissions

In [ ]:
External_costs['PM2.5 (Health - VOLY) [Euro]'] = Emissions_GDP_Population_MDC['PM2.5 [t]'] * Emissions_GDP_Population_MDC['Health 1: PM2.5 VOLY [Euro/t]']
External_costs['PM2.5 (Health - VSL) [Euro]'] = Emissions_GDP_Population_MDC['PM2.5 [t]'] * Emissions_GDP_Population_MDC['Health 1: PM2.5 VSL [Euro/t]']

## External costs due to coarse particulate matter (PM10) emissions

In [ ]:
External_costs['PM10 (Health - VOLY) [Euro]'] = Emissions_GDP_Population_MDC['PM10 [t]'] * Emissions_GDP_Population_MDC['Health 1: PM10 VOLY [Euro/t]']
External_costs['PM10 (Health - VSL) [Euro]'] = Emissions_GDP_Population_MDC['PM10 [t]'] * Emissions_GDP_Population_MDC['Health 1: PM10 VSL [Euro/t]']

## External costs due to Sulfur oxides (SOx) emissions

In [ ]:
External_costs['SOx (Health - VOLY) [Euro]'] = Emissions_GDP_Population_MDC['SOx [t]'] * Emissions_GDP_Population_MDC['Health 1: SOx VOLY [Euro/t]']
External_costs['SOx (Health - VSL) [Euro]'] = Emissions_GDP_Population_MDC['SOx [t]'] * Emissions_GDP_Population_MDC['Health 1: SOx VSL [Euro/t]']
External_costs['SOx (Buildings) [Euro]'] = Emissions_GDP_Population_MDC['SOx [t]'] * Emissions_GDP_Population_MDC['Buildings: Sox [Euro/t]']
External_costs['SOx (Total - VOLY) [Euro]'] = External_costs['SOx (Health - VOLY) [Euro]'] + External_costs['SOx (Buildings) [Euro]']
External_costs['SOx (Total - VSL) [Euro]'] = External_costs['SOx (Health - VSL) [Euro]'] + External_costs['SOx (Buildings) [Euro]']

## External costs due to Non-methane volatile organic compound (NMVOC) emissions

In [ ]:
External_costs['NMVOC (Health - VOLY) [Euro]'] = Emissions_GDP_Population_MDC['NMVOC [t]'] * Emissions_GDP_Population_MDC['Health 1: NMVOC VOLY [Euro/t]']
External_costs['NMVOC (Health - VSL) [Euro]'] = Emissions_GDP_Population_MDC['NMVOC [t]'] * Emissions_GDP_Population_MDC['Health 1: NMVOC VSL [Euro/t]']
External_costs['NMVOC (Forests & Crops) [Euro]'] = Emissions_GDP_Population_MDC['NMVOC [t]'] * (Emissions_GDP_Population_MDC['Crops: NMVOC [Euro/t]'] + Emissions_GDP_Population_MDC['Forests: NMVOC [Euro/t]'])
External_costs['NMVOC (Forests) [Euro]'] = Emissions_GDP_Population_MDC['NMVOC [t]'] * Emissions_GDP_Population_MDC['Forests: NMVOC [Euro/t]']
External_costs['NMVOC (Crops) [Euro]'] = Emissions_GDP_Population_MDC['NMVOC [t]'] * Emissions_GDP_Population_MDC['Crops: NMVOC [Euro/t]']
External_costs['NMVOC (Total - VOLY) [Euro]'] = External_costs['NMVOC (Health - VOLY) [Euro]'] + External_costs['NMVOC (Forests & Crops) [Euro]']
External_costs['NMVOC (Total - VSL) [Euro]'] = External_costs['NMVOC (Health - VSL) [Euro]'] + External_costs['NMVOC (Forests & Crops) [Euro]']

## External costs due to Ammonia (NH3) emissions

In [ ]:
External_costs['NH3 (Health - VOLY) [Euro]'] = Emissions_GDP_Population_MDC['NH3 [t]'] * Emissions_GDP_Population_MDC['Health 1: NH3 VOLY [Euro/t]']
External_costs['NH3 (Health - VSL) [Euro]'] = Emissions_GDP_Population_MDC['NH3 [t]'] * Emissions_GDP_Population_MDC['Health 1: NH3 VSL [Euro/t]']
External_costs['NH3 (Ecosystems) [Euro]'] = Emissions_GDP_Population_MDC['NH3 [t]'] * Emissions_GDP_Population_MDC['Ecosystems: NH3 [Euro/t]']
External_costs['NH3 (Total - VOLY) [Euro]'] = External_costs['NH3 (Health - VOLY) [Euro]'] + External_costs['NH3 (Ecosystems) [Euro]']
External_costs['NH3 (Total - VSL) [Euro]'] = External_costs['NH3 (Health - VSL) [Euro]'] + External_costs['NH3 (Ecosystems) [Euro]']

## External costs due to heavy metals

In [ ]:
External_costs['Pb (Total) [Euro]'] = Emissions_GDP_Population_MDC['Pb [t]'] * Emissions_GDP_Population_MDC['Pb cost [Euro/t]']
External_costs['Hg (Total) [Euro]'] = Emissions_GDP_Population_MDC['Hg [t]'] * Emissions_GDP_Population_MDC['Hg cost [Euro/t]']
External_costs['Cd (Total) [Euro]'] = Emissions_GDP_Population_MDC['Cd [t]'] * Emissions_GDP_Population_MDC['Cd cost [Euro/t]']
External_costs['As (Total) [Euro]'] = Emissions_GDP_Population_MDC['As [t]'] * Emissions_GDP_Population_MDC['As cost [Euro/t]']
External_costs['Ni (Total) [Euro]'] = Emissions_GDP_Population_MDC['Ni [t]'] * Emissions_GDP_Population_MDC['Ni cost [Euro/t]']
External_costs['Cr (Total) [Euro]'] = Emissions_GDP_Population_MDC['Cr [t]'] * Emissions_GDP_Population_MDC['Cr cost [Euro/t]']

## Total impact on health

In [ ]:
External_costs['Health - VOLY (Total) [Euro]'] = (
    External_costs['Nox (Health - VOLY) [Euro]'] +
    External_costs['PM2.5 (Health - VOLY) [Euro]'] +
    External_costs['PM10 (Health - VOLY) [Euro]'] +
    External_costs['SOx (Health - VOLY) [Euro]'] +
    External_costs['NMVOC (Health - VOLY) [Euro]'] +
    External_costs['NH3 (Health - VOLY) [Euro]']
)

External_costs['Health - VSL (Total) [Euro]'] = (
    External_costs['Nox (Health - VSL) [Euro]'] +
    External_costs['PM2.5 (Health - VSL) [Euro]'] +
    External_costs['PM10 (Health - VSL) [Euro]'] +
    External_costs['SOx (Health - VSL) [Euro]'] +
    External_costs['NMVOC (Health - VSL) [Euro]'] +
    External_costs['NH3 (Health - VSL) [Euro]']
)

## Total impact on Crops, Forests, Buildings & Ecosystems

In [ ]:
External_costs['Crops (Total) [Euro]'] = External_costs['Nox (Crops) [Euro]'] + External_costs['NMVOC (Crops) [Euro]']
External_costs['Forests (Total) [Euro]'] = External_costs['Nox (Forests) [Euro]'] + External_costs['NMVOC (Forests) [Euro]']
External_costs['Buildings (Total) [Euro]'] = External_costs['Nox (Buildings) [Euro]'] + External_costs['SOx (Buildings) [Euro]']
External_costs['Ecosystems (Total) [Euro]'] = External_costs['Nox (Ecosystems) [Euro]'] + External_costs['NH3 (Ecosystems) [Euro]']

## Air Pollutants

In [ ]:
External_costs['Air Pollutants - VOLY [Euro]'] = (
    External_costs['Health - VOLY (Total) [Euro]'] +
    External_costs['Crops (Total) [Euro]'] +
    External_costs['Forests (Total) [Euro]'] +
    External_costs['Buildings (Total) [Euro]'] +
    External_costs['Ecosystems (Total) [Euro]']
)

External_costs['Air Pollutants - VSL [Euro]'] = (
    External_costs['Health - VSL (Total) [Euro]'] +
    External_costs['Crops (Total) [Euro]'] +
    External_costs['Forests (Total) [Euro]'] +
    External_costs['Buildings (Total) [Euro]'] +
    External_costs['Ecosystems (Total) [Euro]']
)

## Heavy Metals

In [ ]:
External_costs['Heavy Metals (Total) [Euro]'] = (
    External_costs['Pb (Total) [Euro]'] +
    External_costs['Hg (Total) [Euro]'] +
    External_costs['Cd (Total) [Euro]'] +
    External_costs['As (Total) [Euro]'] +
    External_costs['Ni (Total) [Euro]'] +
    External_costs['Cr (Total) [Euro]']
)

## Total External Costs

In [ ]:
def Total(column_list, column_name, EXT_COST):
  EXT_COST[column_name] = [0] * len(EXT_COST)
  for column in column_list:
    EXT_COST[column_name] += EXT_COST[column]
  return EXT_COST

default_list = ['Crops (Total) [Euro]','Forests (Total) [Euro]','Buildings (Total) [Euro]','Ecosystems (Total) [Euro]','Heavy Metals (Total) [Euro]']
GHG_list = ['GHG: Territorial & Low scenario [Euro]',
       'GHG: Territorial & Central scenario [Euro]',
       'GHG: Territorial & High scenario [Euro]',
       'GHG: Residential & Low scenario [Euro]',
       'GHG: Residential & Central scenario [Euro]',
       'GHG: Residential & High scenario [Euro]',
       'GHG: Footprint & Low scenario [Euro]',
       'GHG: Footprint & Central scenario [Euro]',
       'GHG: Footprint & High scenario [Euro]']
Health_list = ['Health - VOLY (Total) [Euro]', 'Health - VSL (Total) [Euro]']

for GHG_scenario in GHG_list:
    for Health_scenario in Health_list:
      column_list = default_list + [GHG_scenario, Health_scenario ]
      column_name = f'External costs ({GHG_scenario.split()[1]} & {GHG_scenario.split()[3]} GHG with {Health_scenario.split()[2]}) scenario [Euro]'
      External_costs = Total(column_list, column_name, External_costs)

## Test of the importance of As, Ni and Cr

In [ ]:
column_list_test = ['External costs (Territorial & Low GHG with VOLY) scenario [Euro]',
       'External costs (Territorial & Low GHG with VSL) scenario [Euro]',
       'External costs (Territorial & Central GHG with VOLY) scenario [Euro]',
       'External costs (Territorial & Central GHG with VSL) scenario [Euro]',
       'External costs (Territorial & High GHG with VOLY) scenario [Euro]',
       'External costs (Territorial & High GHG with VSL) scenario [Euro]',
       'External costs (Residential & Low GHG with VOLY) scenario [Euro]',
       'External costs (Residential & Low GHG with VSL) scenario [Euro]',
       'External costs (Residential & Central GHG with VOLY) scenario [Euro]',
       'External costs (Residential & Central GHG with VSL) scenario [Euro]',
       'External costs (Residential & High GHG with VOLY) scenario [Euro]',
       'External costs (Residential & High GHG with VSL) scenario [Euro]',
       'External costs (Footprint & Low GHG with VOLY) scenario [Euro]',
       'External costs (Footprint & Low GHG with VSL) scenario [Euro]',
       'External costs (Footprint & Central GHG with VOLY) scenario [Euro]',
       'External costs (Footprint & Central GHG with VSL) scenario [Euro]',
       'External costs (Footprint & High GHG with VOLY) scenario [Euro]',
       'External costs (Footprint & High GHG with VSL) scenario [Euro]']
for column in column_list_test:
  new_column = 'Proportion in % - ' + column
  External_costs[new_column] = 100*(External_costs['As (Total) [Euro]'] + External_costs['Ni (Total) [Euro]'] + External_costs['Cr (Total) [Euro]'])/External_costs[column]
  print(f'For {column} maximum: {External_costs[new_column].max():0.2f}% mean: {External_costs[new_column].mean():0.2f}%')

For External costs (Territorial & Low GHG with VOLY) scenario [Euro] maximum: 3.72% mean: 0.42%
For External costs (Territorial & Low GHG with VSL) scenario [Euro] maximum: 2.53% mean: 0.22%
For External costs (Territorial & Central GHG with VOLY) scenario [Euro] maximum: 3.10% mean: 0.36%
For External costs (Territorial & Central GHG with VSL) scenario [Euro] maximum: 2.22% mean: 0.20%
For External costs (Territorial & High GHG with VOLY) scenario [Euro] maximum: 2.78% mean: 0.17%
For External costs (Territorial & High GHG with VSL) scenario [Euro] maximum: 1.60% mean: 0.12%
For External costs (Residential & Low GHG with VOLY) scenario [Euro] maximum: 2.33% mean: 0.27%
For External costs (Residential & Low GHG with VSL) scenario [Euro] maximum: 1.59% mean: 0.14%
For External costs (Residential & Central GHG with VOLY) scenario [Euro] maximum: 2.18% mean: 0.21%
For External costs (Residential & Central GHG with VSL) scenario [Euro] maximum: 1.52% mean: 0.12%
For External costs (Residen

## External Costs as % of GDP and per capita

In [ ]:
Emissions_GDP_Population_MDC['GDP [Euro]'] = Emissions_GDP_Population_MDC['GDP [MIO_EURO]']*1e6

In [ ]:
df_GDP_POP_EC = Emissions_GDP_Population_MDC.loc[:,['countries','Year','GDP [Euro]','Population']]
External_costs = pd.merge(External_costs, df_GDP_POP_EC, on=['countries','Year'], how='inner')

In [ ]:
column_list = ['External costs (Territorial & Low GHG with VOLY) scenario [Euro]',
       'External costs (Territorial & Low GHG with VSL) scenario [Euro]',
       'External costs (Territorial & Central GHG with VOLY) scenario [Euro]',
       'External costs (Territorial & Central GHG with VSL) scenario [Euro]',
       'External costs (Territorial & High GHG with VOLY) scenario [Euro]',
       'External costs (Territorial & High GHG with VSL) scenario [Euro]',
       'External costs (Residential & Low GHG with VOLY) scenario [Euro]',
       'External costs (Residential & Low GHG with VSL) scenario [Euro]',
       'External costs (Residential & Central GHG with VOLY) scenario [Euro]',
       'External costs (Residential & Central GHG with VSL) scenario [Euro]',
       'External costs (Residential & High GHG with VOLY) scenario [Euro]',
       'External costs (Residential & High GHG with VSL) scenario [Euro]',
       'External costs (Footprint & Low GHG with VOLY) scenario [Euro]',
       'External costs (Footprint & Low GHG with VSL) scenario [Euro]',
       'External costs (Footprint & Central GHG with VOLY) scenario [Euro]',
       'External costs (Footprint & Central GHG with VSL) scenario [Euro]',
       'External costs (Footprint & High GHG with VOLY) scenario [Euro]',
       'External costs (Footprint & High GHG with VSL) scenario [Euro]']

per_capita_columns = []
percentage_columns = []

for external_costs in column_list:
    # Create column names for per capita values
    per_capita_column_name = f'External costs per capita {external_costs.split()[2]} {external_costs.split()[3]} {external_costs.split()[4]} {external_costs.split()[5]} {external_costs.split()[6]} {external_costs.split()[7]} scenario [Euro]'
    # Calculate per capita values and append to the list
    per_capita_values = External_costs[external_costs] / External_costs['Population']
    per_capita_columns.append(pd.DataFrame({per_capita_column_name: per_capita_values}))

    # Create column names for percentage values
    percentage_column_name = f'{external_costs.split()[2]} {external_costs.split()[3]} {external_costs.split()[4]} {external_costs.split()[5]} {external_costs.split()[6]} {external_costs.split()[7]} scenario [%]'
    # Calculate percentage values and append to the list
    percentage_values = (External_costs[external_costs] / External_costs['GDP [Euro]']) * 100
    percentage_columns.append(pd.DataFrame({percentage_column_name: percentage_values}))

# Concatenate all per capita columns
per_capita_df = pd.concat(per_capita_columns, axis=1)

# Concatenate all percentage columns
percentage_df = pd.concat(percentage_columns, axis=1)

# Concatenate the new columns to the original DataFrame
External_costs = pd.concat([External_costs, per_capita_df, percentage_df], axis=1)

# GrDP

## Calculating GrDP

In [ ]:
GrDP = External_costs.loc[:,['countries','Year','GDP [Euro]','Population']]
column_list = ['External costs (Territorial & Low GHG with VOLY) scenario [Euro]',
       'External costs (Territorial & Low GHG with VSL) scenario [Euro]',
       'External costs (Territorial & Central GHG with VOLY) scenario [Euro]',
       'External costs (Territorial & Central GHG with VSL) scenario [Euro]',
       'External costs (Territorial & High GHG with VOLY) scenario [Euro]',
       'External costs (Territorial & High GHG with VSL) scenario [Euro]',
       'External costs (Residential & Low GHG with VOLY) scenario [Euro]',
       'External costs (Residential & Low GHG with VSL) scenario [Euro]',
       'External costs (Residential & Central GHG with VOLY) scenario [Euro]',
       'External costs (Residential & Central GHG with VSL) scenario [Euro]',
       'External costs (Residential & High GHG with VOLY) scenario [Euro]',
       'External costs (Residential & High GHG with VSL) scenario [Euro]',
       'External costs (Footprint & Low GHG with VOLY) scenario [Euro]',
       'External costs (Footprint & Low GHG with VSL) scenario [Euro]',
       'External costs (Footprint & Central GHG with VOLY) scenario [Euro]',
       'External costs (Footprint & Central GHG with VSL) scenario [Euro]',
       'External costs (Footprint & High GHG with VOLY) scenario [Euro]',
       'External costs (Footprint & High GHG with VSL) scenario [Euro]']
for external_costs in column_list:
  column_name = f'GrDP {external_costs.split()[2]} {external_costs.split()[3]} {external_costs.split()[4]} {external_costs.split()[5]} {external_costs.split()[6]} {external_costs.split()[7]} scenario [Euro]'
  GrDP[column_name] = External_costs['GDP [Euro]'] - External_costs[external_costs]

## Calculating GrDP per capita

In [ ]:
column_list = ['GrDP (Territorial & Low GHG with VOLY) scenario [Euro]',
       'GrDP (Territorial & Low GHG with VSL) scenario [Euro]',
       'GrDP (Territorial & Central GHG with VOLY) scenario [Euro]',
       'GrDP (Territorial & Central GHG with VSL) scenario [Euro]',
       'GrDP (Territorial & High GHG with VOLY) scenario [Euro]',
       'GrDP (Territorial & High GHG with VSL) scenario [Euro]',
       'GrDP (Residential & Low GHG with VOLY) scenario [Euro]',
       'GrDP (Residential & Low GHG with VSL) scenario [Euro]',
       'GrDP (Residential & Central GHG with VOLY) scenario [Euro]',
       'GrDP (Residential & Central GHG with VSL) scenario [Euro]',
       'GrDP (Residential & High GHG with VOLY) scenario [Euro]',
       'GrDP (Residential & High GHG with VSL) scenario [Euro]',
       'GrDP (Footprint & Low GHG with VOLY) scenario [Euro]',
       'GrDP (Footprint & Low GHG with VSL) scenario [Euro]',
       'GrDP (Footprint & Central GHG with VOLY) scenario [Euro]',
       'GrDP (Footprint & Central GHG with VSL) scenario [Euro]',
       'GrDP (Footprint & High GHG with VOLY) scenario [Euro]',
       'GrDP (Footprint & High GHG with VSL) scenario [Euro]']
for GrDP_column in column_list:
  column_name = f'GrDP per capita {GrDP_column.split()[1]} {GrDP_column.split()[2]} {GrDP_column.split()[3]} {GrDP_column.split()[4]} {GrDP_column.split()[5]} {GrDP_column.split()[6]} scenario [Euro]'
  GrDP[column_name] = GrDP[GrDP_column]/GrDP['Population']
GrDP['GDP per capita [Euro]'] = GrDP['GDP [Euro]']/GrDP['Population']

# Decoupling

In [ ]:
Data_for_decoupling = pd.merge(Emissions, External_costs, on=['countries','Year'], how='inner')

## Set reference year

In [ ]:
# We check which years have no missing values
for year in range(1990,2022):
  sums_list = list(Data_for_decoupling.loc[Data_for_decoupling.Year >= year].isna().sum())
  if all(el == 0 for el in sums_list):
    print(f'{year} is good')
  else:
    print(f'For {year} we have missing values')

For 1990 we have missing values
For 1991 we have missing values
For 1992 we have missing values
For 1993 we have missing values
For 1994 we have missing values
For 1995 we have missing values
For 1996 we have missing values
For 1997 we have missing values
For 1998 we have missing values
For 1999 we have missing values
For 2000 we have missing values
For 2001 we have missing values
For 2002 we have missing values
For 2003 we have missing values
For 2004 we have missing values
For 2005 we have missing values
For 2006 we have missing values
For 2007 we have missing values
2008 is good
2009 is good
2010 is good
2011 is good
2012 is good
2013 is good
2014 is good
2015 is good
2016 is good
2017 is good
2018 is good
2019 is good
2020 is good
2021 is good


In [ ]:
Reference_Year = 2011

## OECD decoupling indicator

In [ ]:
####### Function definition #######

def summing_emissions(column_list, df, new_column):
    df[new_column] = df[column_list].sum(axis=1)


def normalisation(column, df_results, df_calc, ref_year):
    new_column = column.rsplit(maxsplit=1)[0]
    norm_values = []

    for country in df_results['countries'].unique():
        value_0 = df_calc.loc[(df_calc.countries == country) & (df_calc.Year == ref_year), column].mean()
        if value_0 != 0:
            for year in df_results.loc[df_results.Year > ref_year, 'Year'].unique():
                value_t = df_calc.loc[(df_calc.countries == country) & (df_calc.Year == year), column].mean()
                norm_values.append((country, year, value_t / value_0))

    # Create a DataFrame from the list and merge it back
    norm_df = pd.DataFrame(norm_values, columns=['countries', 'Year', new_column])
    df_results = df_results.merge(norm_df, on=['countries', 'Year'], how='left')

    return df_results


def decoupling_OECD(column, column_WRT, df_results, df_calc):
    new_column = f"{column}_{column_WRT}"
    decoupling_values = []

    for country in df_results['countries'].unique():
        for year in df_results['Year'].unique():
            Et0 = df_calc.loc[(df_calc['countries'] == country) & (df_calc['Year'] == year), column].mean()
            Yt0 = df_calc.loc[(df_calc['countries'] == country) & (df_calc['Year'] == year), column_WRT].mean()
            if Yt0 != 0:
                decoupling_values.append((country, year, 1 - Et0 / Yt0))

    # Create a DataFrame from the list and merge it back
    decoupling_df = pd.DataFrame(decoupling_values, columns=['countries', 'Year', new_column])
    df_results = df_results.merge(decoupling_df, on=['countries', 'Year'], how='left')

    return df_results



####### Main #######

# Define the columns for heavy metals and air pollutants
heavy_metals = ['As [t]', 'Pb [t]', 'Cd [t]', 'Hg [t]', 'Cr [t]', 'Ni [t]']
air_pollutants = ['NOx [t]', 'SOx [t]', 'NH3 [t]', 'PM2.5 [t]', 'PM10 [t]', 'NMVOC [t]']

# Summing emissions
summing_emissions(heavy_metals, Data_for_decoupling, 'Heavy Metals Emissions [t]')
summing_emissions(air_pollutants, Data_for_decoupling, 'Air Pollutants Emissions [t]')

# Prepare DataFrame for ratios
Ratio_for_decoupling = Data_for_decoupling.loc[Data_for_decoupling.Year > Reference_Year, ['countries', 'Year']]

# Normalize data
for column in Data_for_decoupling.columns[2:]:
    Ratio_for_decoupling = normalisation(column, Ratio_for_decoupling, Data_for_decoupling, Reference_Year)

# Prepare DataFrame for decoupling
Decoupling_OECD = Ratio_for_decoupling.loc[:, ['countries', 'Year']]

# Calculate decoupling metrics
for column in Ratio_for_decoupling.columns[2:]:
    Decoupling_OECD = decoupling_OECD(column, 'GDP', Decoupling_OECD, Ratio_for_decoupling)
    Decoupling_OECD = decoupling_OECD(column, 'Population', Decoupling_OECD, Ratio_for_decoupling)

<ipython-input-29-98af0f8c4ccc>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column] = df[column_list].sum(axis=1)
<ipython-input-29-98af0f8c4ccc>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column] = df[column_list].sum(axis=1)


## Intensity decoupling factor

In [ ]:
####### Function definition #######

def Relative_Difference(column, df_results, df_calc, ref_year):
    new_column = column.rsplit(maxsplit=1)[0]
    rel_diff_values = []

    for country in df_results['countries'].unique():
        value_0 = df_calc.loc[(df_calc.countries == country) & (df_calc.Year == ref_year), column].mean()
        if value_0 != 0:
            for year in df_results.loc[df_results.Year > ref_year, 'Year'].unique():
                value_t = df_calc.loc[(df_calc.countries == country) & (df_calc.Year == year), column].mean()
                rel_diff_values.append((country, year, (value_t - value_0) / value_0))

    # Create a DataFrame from the list and merge it back
    rel_diff_df = pd.DataFrame(rel_diff_values, columns=['countries', 'Year', new_column])
    df_results = df_results.merge(rel_diff_df, on=['countries', 'Year'], how='left')

    return df_results


def IntensityFactorDecoupling(column, column_WRT, df_results, df_calc):
    new_column = f"{column}_{column_WRT}"
    intensity_values = []

    for country in df_results['countries'].unique():
        for year in df_results['Year'].unique():
            gE = df_calc.loc[(df_calc.countries == country) & (df_calc.Year == year), column].mean()
            gY = df_calc.loc[(df_calc.countries == country) & (df_calc.Year == year), column_WRT].mean()
            if gY != 0:
                intensity_values.append((country, year, -gE/gY if gY > 0 else gE/gY))

    # Create a DataFrame from the list and merge it back
    intensity_df = pd.DataFrame(intensity_values, columns=['countries', 'Year', new_column])
    df_results = df_results.merge(intensity_df, on=['countries', 'Year'], how='left')

    return df_results

####### Main #######

# Prepare DataFrame for relative differences
RelDifference_for_decoupling = Data_for_decoupling.loc[Data_for_decoupling.Year > Reference_Year, ['countries', 'Year']]

# Calculate relative differences
for column in Data_for_decoupling.columns[2:]:
    RelDifference_for_decoupling = Relative_Difference(column, RelDifference_for_decoupling, Data_for_decoupling, Reference_Year)

# Prepare DataFrame for intensity factor decoupling
Decoupling_IntensityFactor = RelDifference_for_decoupling.loc[:, ['countries', 'Year']]

# Calculate intensity factor decoupling
for column in RelDifference_for_decoupling.columns[2:]:
    Decoupling_IntensityFactor = IntensityFactorDecoupling(column, 'GDP', Decoupling_IntensityFactor, RelDifference_for_decoupling)
    Decoupling_IntensityFactor = IntensityFactorDecoupling(column, 'Population', Decoupling_IntensityFactor, RelDifference_for_decoupling)

In [ ]:
list(Decoupling_IntensityFactor)

['countries',
 'Year',
 'Territorial GHG emissions_GDP',
 'Territorial GHG emissions_Population',
 'Residential GHG emissions_GDP',
 'Residential GHG emissions_Population',
 'Transfer GHG emissions_GDP',
 'Transfer GHG emissions_Population',
 'Footprint GHG emissions_GDP',
 'Footprint GHG emissions_Population',
 'As_GDP',
 'As_Population',
 'Pb_GDP',
 'Pb_Population',
 'NOx_GDP',
 'NOx_Population',
 'SOx_GDP',
 'SOx_Population',
 'NH3_GDP',
 'NH3_Population',
 'PM2.5_GDP',
 'PM2.5_Population',
 'PM10_GDP',
 'PM10_Population',
 'NMVOC_GDP',
 'NMVOC_Population',
 'Cd_GDP',
 'Cd_Population',
 'Hg_GDP',
 'Hg_Population',
 'Cr_GDP',
 'Cr_Population',
 'Ni_GDP',
 'Ni_Population',
 'GHG: Territorial & Low scenario_GDP',
 'GHG: Territorial & Low scenario_Population',
 'GHG: Territorial & Central scenario_GDP',
 'GHG: Territorial & Central scenario_Population',
 'GHG: Territorial & High scenario_GDP',
 'GHG: Territorial & High scenario_Population',
 'GHG: Residential & Low scenario_GDP',
 'GHG:

In [ ]:
list(RelDifference_for_decoupling.columns)

['countries',
 'Year',
 'Territorial GHG emissions',
 'Residential GHG emissions',
 'Transfer GHG emissions',
 'Footprint GHG emissions',
 'As',
 'Pb',
 'NOx',
 'SOx',
 'NH3',
 'PM2.5',
 'PM10',
 'NMVOC',
 'Cd',
 'Hg',
 'Cr',
 'Ni',
 'GHG: Territorial & Low scenario',
 'GHG: Territorial & Central scenario',
 'GHG: Territorial & High scenario',
 'GHG: Residential & Low scenario',
 'GHG: Residential & Central scenario',
 'GHG: Residential & High scenario',
 'GHG: Footprint & Low scenario',
 'GHG: Footprint & Central scenario',
 'GHG: Footprint & High scenario',
 'Nox (Health - VOLY)',
 'Nox (Health - VSL)',
 'Nox (Crops, Forests, Buildings & Ecosystems)',
 'Nox (Crops)',
 'Nox (Forests)',
 'Nox (Buildings)',
 'Nox (Ecosystems)',
 'Nox (Total - VOLY)',
 'Nox (Total - VSL)',
 'PM2.5 (Health - VOLY)',
 'PM2.5 (Health - VSL)',
 'PM10 (Health - VOLY)',
 'PM10 (Health - VSL)',
 'SOx (Health - VOLY)',
 'SOx (Health - VSL)',
 'SOx (Buildings)',
 'SOx (Total - VOLY)',
 'SOx (Total - VSL)',
 'NMVO

# Downloading the datasets

In [ ]:
Emissions.to_csv('Emissions.csv', index=False)
External_costs.to_csv('ExternalCosts.csv', index=False)
GrDP.to_csv('GrDP.csv', index=False)
Decoupling_OECD.to_csv('Decoupling_OECD_Indicator.csv', index=False)
Decoupling_IntensityFactor.to_csv('Decoupling_Intensity_Factor.csv', index=False)
RelDifference_for_decoupling.to_csv('Decoupling_Relative_Difference.csv', index=False)